<a href="https://colab.research.google.com/github/jordanmsouza/TechChallenge_Fase3_Grupo4/blob/main/TechChallenge_Fase3_Grupo4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Montando o Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Instalação de dependências:

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-b5fbly2i/unsloth_a81326b4ed9a4312abbc548a7a9f7231
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-b5fbly2i/unsloth_a81326b4ed9a4312abbc548a7a9f7231
  Resolved https://github.com/unslothai/unsloth.git to commit 0fbbdfc091fc1a3b1c09b752794963681d10fad2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 30.2 MB/s eta 0:00:00
   

### Convertendo trn.json para CSV

In [4]:
import json
import pandas as pd

# Função para realizar o pré-processamento
def preprocess_amazon_data(input_json_path, output_csv_path):
    # Carregar o arquivo JSON
    with open(input_json_path, 'r') as file:
        data = [json.loads(line) for line in file]

    # Criar um DataFrame com as colunas relevantes
    df = pd.DataFrame(data, columns=['title', 'content'])

    # Salvar o DataFrame em um arquivo CSV
    df.to_csv(output_csv_path, index=False)
    print(f"Arquivo CSV gerado com sucesso em: {output_csv_path}")

# Exemplo de uso
input_json_path = '/content/drive/MyDrive/trn.json/trn.json'  # Altere para o caminho correto do arquivo JSON
output_csv_path = '/content/drive/MyDrive/trn.json/amazon_product_data.csv'  # Altere para o caminho desejado de saída

preprocess_amazon_data(input_json_path, output_csv_path)


Arquivo CSV gerado com sucesso em: /content/drive/MyDrive/trn.json/amazon_product_data.csv
